In [2]:
import pandas as pd
import numpy as np
import os
import base64
import requests
import urllib.parse
import pyarrow as pa
import pyarrow.parquet as pq
import boto3

In [3]:
AWS_KEY = os.getenv('AWS_KEY')
AWS_SECRET = os.getenv('AWS_SECRET')
BUCKET_NAME = os.getenv('BUCKET_NAME')
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

In [4]:
API_KEY = os.getenv('TULIP_TABLE_API_KEY')
API_SECRET = os.getenv('TULIP_TABLE_API_SECRET')

In [5]:
API_CREDENTIALS = 'Basic '+base64.b64encode((API_KEY+":"+API_SECRET).encode('ascii')).decode('ascii')

In [6]:
base_url = 'https://william.tulip.co/api/v3/tables/'
table = 'fXsbaj23HEPFRnTyZ/records'
limit = 'limit=100'
sort_array = '[{"sortBy":"wrwyx_datetime_downtime_start","sortDir":"desc"}]'
sort_query = 'sortOptions='+urllib.parse.quote(sort_array)
SQL = f'?{limit}&{sort_query}'
query = base_url + table + SQL

In [7]:
header = {"Authorization" : API_CREDENTIALS}
res = requests.get(query, headers=header)
print(res)
print(len(res.json()))

<Response [200]>
6


In [8]:
df = pd.json_normalize(res.json())
df.head()

,_createdAt,_sequenceNumber,_updatedAt,id,ikkuy_location,jlplh_reason,kcldd_date_downtime_start,tsclt_description,vkves_downtime,wrwyx_datetime_downtime_start
0,2023-05-31T18:54:12Z,6,2023-05-31T18:54:13Z,Machine Shop-1685559252,Machine Shop,Reactive Maintenance,2023-05-30T23:00:00-05:00,Update \n,80,2023-05-31T13:52:53-05:00
1,2023-05-31T18:52:40Z,5,2023-05-31T18:52:41Z,Machine Shop-1685559160,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Aa,347622,2023-05-27T13:18:59-05:00
2,2023-05-27T18:02:09Z,3,2023-05-27T18:02:09Z,Machine Shop-1685210529,Machine Shop,Machine breakdown,2023-05-26T23:00:00-05:00,Tooling damaged. Critical. \nneed Katey’s pers...,84,2023-05-27T13:00:45-05:00
3,2023-05-27T17:57:32Z,2,2023-05-27T17:57:32Z,Machine Shop-1685210251,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Beb\n\n,88,2023-05-27T12:56:04-05:00
4,2023-05-27T18:18:38Z,4,2023-05-27T18:18:39Z,Assembly Line-1685211517,Assembly Line,Planned downtime,2023-05-26T23:00:00-05:00,Dd,46736,2023-05-27T00:19:42-05:00


In [9]:
df.shape

(6, 10)

In [10]:
df.columns

Index(['_createdAt', '_sequenceNumber', '_updatedAt', 'id', 'ikkuy_location',
       'jlplh_reason', 'kcldd_date_downtime_start', 'tsclt_description',
       'vkves_downtime', 'wrwyx_datetime_downtime_start'],
      dtype='object')

In [11]:
clean_columns = [column if column in ['_createdAt', '_sequenceNumber', '_updatedAt', 'id'] else column[6:] for column in df.columns]
df.columns = clean_columns
df.head()

,_createdAt,_sequenceNumber,_updatedAt,id,location,reason,date_downtime_start,description,downtime,datetime_downtime_start
0,2023-05-31T18:54:12Z,6,2023-05-31T18:54:13Z,Machine Shop-1685559252,Machine Shop,Reactive Maintenance,2023-05-30T23:00:00-05:00,Update \n,80,2023-05-31T13:52:53-05:00
1,2023-05-31T18:52:40Z,5,2023-05-31T18:52:41Z,Machine Shop-1685559160,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Aa,347622,2023-05-27T13:18:59-05:00
2,2023-05-27T18:02:09Z,3,2023-05-27T18:02:09Z,Machine Shop-1685210529,Machine Shop,Machine breakdown,2023-05-26T23:00:00-05:00,Tooling damaged. Critical. \nneed Katey’s pers...,84,2023-05-27T13:00:45-05:00
3,2023-05-27T17:57:32Z,2,2023-05-27T17:57:32Z,Machine Shop-1685210251,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Beb\n\n,88,2023-05-27T12:56:04-05:00
4,2023-05-27T18:18:38Z,4,2023-05-27T18:18:39Z,Assembly Line-1685211517,Assembly Line,Planned downtime,2023-05-26T23:00:00-05:00,Dd,46736,2023-05-27T00:19:42-05:00


In [12]:
df.to_parquet("data/downtime_history.parquet")

In [14]:
s3.Bucket(BUCKET_NAME).upload_file(Filename='data/downtime_history.parquet', Key='downtime_history/downtime_history.parquet')